In [0]:
# ============================================================
# Dimensão: CLIENTES (Bronze)
# Geração de dados sintéticos com Faker pt_BR e gravação como Tabela Managed em UC
# Padrão de namespace: <catalog>.<schema>.dim_clientes
#
# Parâmetros esperados (dbutils.widgets):
#   - catalog (default: prd)
#   - schema  (default: b_dm_callcenter)
#   - table   (default: dim_clientes)
#   - qtd_clientes (default: 10000)
# ============================================================

from pyspark.sql import functions as F
from datetime import datetime
import random
from faker import Faker

# ---------------- Parâmetros ----------------
try:
    catalog = dbutils.widgets.get("catalog")
except:
    dbutils.widgets.text("catalog", "prd")
    catalog = dbutils.widgets.get("catalog")

try:
    schema = dbutils.widgets.get("schema")
except:
    dbutils.widgets.text("schema", "b_dm_callcenter")
    schema = dbutils.widgets.get("schema")

try:
    table = dbutils.widgets.get("table")
except:
    dbutils.widgets.text("table", "dim_clientes")
    table = dbutils.widgets.get("table")

try:
    qtd_clientes = int(dbutils.widgets.get("qtd_clientes"))
except:
    dbutils.widgets.text("qtd_clientes", "1000")
    qtd_clientes = int(dbutils.widgets.get("qtd_clientes"))

fake = Faker("pt_BR")

segmentos = ["Varejo", "Prime", "Private", "PJ", "Governo"]
ufs = ['SP', 'RJ', 'MG', 'BA', 'RS', 'PR', 'SC', 'PE', 'CE', 'DF']

# ---------------- Geração dos dados ----------------
clientes = []
for i in range(1, qtd_clientes + 1):
    clientes.append({
        "id_cliente": i,
        "nome": fake.name(),
        "cpf": fake.ssn(),
        "email": fake.email(),
        "data_nascimento": fake.date_of_birth(minimum_age=18, maximum_age=80),
        "sexo": random.choice(["Masculino", "Feminino", "Outro"]),
        "estado": random.choice(ufs),
        "cidade": fake.city(),
        "segmento": random.choice(segmentos),
        "dt_cadastro": datetime.now(),
    })

df = spark.createDataFrame(clientes) \
    .withColumn("dat_ref_carga", F.current_date()) \
    .withColumn("ingestion_ts", F.current_timestamp())

db_table = f"{catalog}.{schema}.{table}"

(
    df.write
      .format("delta")
      .mode("overwrite")
      .option("overwriteSchema", "true")
      .saveAsTable(db_table) 
)

spark.sql(f"OPTIMIZE {db_table}")

print(f"[OK] Dimensão criada/atualizada: {db_table}")
